**Loaded packages**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import KNNImputer

**Load Data**

In [ ]:
train = pd.read_csv("../data/raw/train.csv")
test = pd.read_csv("../data/raw/test.csv")

In [ ]:
train.head()

,Trip_ID,Trip_Distance,Type_of_Cab,Customer_Since_Months,Life_Style_Index,Confidence_Life_Style_Index,Destination_Type,Customer_Rating,Cancellation_Last_1Month,Var1,Var2,Var3,Gender,Surge_Pricing_Type
0,T0005689460,6.77,B,1.0,2.42769,A,A,3.90500,0,40.0,46,60,Female,2
1,T0005689461,29.47,B,10.0,2.78245,B,A,3.45000,0,38.0,56,78,Male,2
2,T0005689464,41.58,NaN,10.0,NaN,NaN,E,3.50125,2,NaN,56,77,Male,2
3,T0005689465,61.56,C,10.0,NaN,NaN,A,3.45375,0,NaN,52,74,Male,3
4,T0005689467,54.95,C,10.0,3.03453,B,A,3.40250,4,51.0,49,102,Male,2


In [ ]:
test.head()

,Trip_ID,Trip_Distance,Type_of_Cab,Customer_Since_Months,Life_Style_Index,Confidence_Life_Style_Index,Destination_Type,Customer_Rating,Cancellation_Last_1Month,Var1,Var2,Var3,Gender
0,T0005689459,9.44,A,10.0,2.57438,A,B,3.68000,2,NaN,46,63,Male
1,T0005689462,32.15,B,10.0,2.85143,A,A,1.59375,0,65.0,49,80,Female
2,T0005689463,10.38,C,4.0,2.70530,A,D,4.50500,0,NaN,47,74,Male
3,T0005689466,14.94,NaN,6.0,2.48159,C,E,4.53000,0,63.0,43,54,Male
4,T0005689468,32.03,B,7.0,2.81598,A,K,4.60125,3,96.0,44,56,Male


Check the count of null values in each variable in Train and Test set

In [ ]:
null_dict_train = {}
for col in train.columns:
    null_dict_train[col] = train[col].isnull().sum()

null_dict_test = {}
for col in test.columns:
    null_dict_test[col] = test[col].isnull().sum()

In [ ]:
print(null_dict_train)

{'Trip_ID': 0, 'Trip_Distance': 0, 'Type_of_Cab': 20210, 'Customer_Since_Months': 5920, 'Life_Style_Index': 20193, 'Confidence_Life_Style_Index': 20193, 'Destination_Type': 0, 'Customer_Rating': 0, 'Cancellation_Last_1Month': 0, 'Var1': 71030, 'Var2': 0, 'Var3': 0, 'Gender': 0, 'Surge_Pricing_Type': 0}


In [ ]:
print(null_dict_test)

{'Trip_ID': 0, 'Trip_Distance': 0, 'Type_of_Cab': 13158, 'Customer_Since_Months': 3966, 'Life_Style_Index': 13327, 'Confidence_Life_Style_Index': 13327, 'Destination_Type': 0, 'Customer_Rating': 0, 'Cancellation_Last_1Month': 0, 'Var1': 46789, 'Var2': 0, 'Var3': 0, 'Gender': 0}


##Preprocess 3: KnnImputer (n = 10 & uniform weights) & Label Encode

###Pre-process

In [ ]:
train['Type_of_Cab'] = train['Type_of_Cab'].replace("A",'1')
train['Type_of_Cab'] = train['Type_of_Cab'].replace("B",'2')
train['Type_of_Cab'] = train['Type_of_Cab'].replace("C",'3')
train['Type_of_Cab'] = train['Type_of_Cab'].replace("D",'4')
train['Type_of_Cab'] = train['Type_of_Cab'].replace("E",'5')
train['Type_of_Cab'] = train['Type_of_Cab'].astype(float).astype('Int64')

test['Type_of_Cab'] = test['Type_of_Cab'].replace("A",'1')
test['Type_of_Cab'] = test['Type_of_Cab'].replace("B",'2')
test['Type_of_Cab'] = test['Type_of_Cab'].replace("C",'3')
test['Type_of_Cab'] = test['Type_of_Cab'].replace("D",'4')
test['Type_of_Cab'] = test['Type_of_Cab'].replace("E",'5')
test['Type_of_Cab'] = test['Type_of_Cab'].astype(float).astype('Int64')


In [ ]:
train['Confidence_Life_Style_Index'] = train['Confidence_Life_Style_Index'].replace("A",'1')
train['Confidence_Life_Style_Index'] = train['Confidence_Life_Style_Index'].replace("B",'2')
train['Confidence_Life_Style_Index'] = train['Confidence_Life_Style_Index'].replace("C",'3')
train['Confidence_Life_Style_Index'] = train['Confidence_Life_Style_Index'].astype(float).astype('Int64')

test['Confidence_Life_Style_Index'] = test['Confidence_Life_Style_Index'].replace("A",'1')
test['Confidence_Life_Style_Index'] = test['Confidence_Life_Style_Index'].replace("B",'2')
test['Confidence_Life_Style_Index'] = test['Confidence_Life_Style_Index'].replace("C",'3')
test['Confidence_Life_Style_Index'] = test['Confidence_Life_Style_Index'].astype(float).astype('Int64')

In [ ]:
dst_type = train['Destination_Type'].unique().tolist()
dest_vals = [0,4,1,2,6,3,5,10,11,7,8,9,12,13]
dest_map = dict(zip(dst_type, dest_vals))

train['Destination_Type'] = train['Destination_Type'].replace(dest_map)
train['Destination_Type'] = train['Destination_Type'].astype(float).astype('Int64')

test['Destination_Type'] = test['Destination_Type'].replace(dest_map)
test['Destination_Type'] = test['Destination_Type'].astype(float).astype('Int64')

In [ ]:
le = LabelEncoder()
trained_le = le.fit(train["Gender"])
train["Gender"] = trained_le.transform(train["Gender"])
test["Gender"] = trained_le.transform(test["Gender"])
train.head(10)

,Trip_ID,Trip_Distance,Type_of_Cab,Customer_Since_Months,Life_Style_Index,Confidence_Life_Style_Index,Destination_Type,Customer_Rating,Cancellation_Last_1Month,Var1,Var2,Var3,Gender,Surge_Pricing_Type
0,T0005689460,6.77,2,1.0,2.42769,1,0,3.90500,0,40.0,46,60,0,2
1,T0005689461,29.47,2,10.0,2.78245,2,0,3.45000,0,38.0,56,78,1,2
2,T0005689464,41.58,<NA>,10.0,NaN,<NA>,4,3.50125,2,NaN,56,77,1,2
3,T0005689465,61.56,3,10.0,NaN,<NA>,0,3.45375,0,NaN,52,74,1,3
4,T0005689467,54.95,3,10.0,3.03453,2,0,3.40250,4,51.0,49,102,1,2
5,T0005689469,19.06,5,10.0,NaN,<NA>,0,2.59750,1,72.0,63,91,1,3
6,T0005689470,29.72,5,10.0,2.83958,3,1,2.97500,1,83.0,50,75,1,2
7,T0005689472,18.44,2,2.0,2.81871,2,0,3.58250,0,103.0,46,63,1,2
8,T0005689473,106.80,3,3.0,NaN,<NA>,0,3.14625,0,NaN,58,92,1,2
9,T0005689474,107.19,4,5.0,3.04467,2,0,2.44375,1,NaN,58,83,1,3


In [ ]:
X = train.drop(["Surge_Pricing_Type","Trip_ID"], axis=1)
y = train.Surge_Pricing_Type
X_test = test.drop(["Trip_ID"], axis=1)
Trip_ID = test.Trip_ID

In [ ]:
# define imputer
imputer = KNNImputer(n_neighbors = 10, weights='uniform')
fitted_imputer = imputer.fit(X)
X = fitted_imputer.transform(X)
X_test = fitted_imputer.transform(X_test)

In [ ]:
X_imputed = pd.DataFrame(X, columns = train.drop(["Surge_Pricing_Type","Trip_ID"], axis=1).columns)
X_test_imputed = pd.DataFrame(X_test, columns = test.drop(["Trip_ID"], axis=1).columns)

In [ ]:
train.to_csv('../data/preprocessed/train_process3.csv', index = False)
test.to_csv('../data/preprocessed/test_process3.csv', index = False)